# Simple 3D Finite Plasticity model from "Continuum Mechanics of Solids"



In [1]:
import numpy as np

import dolfinx

from mpi4py import MPI
from petsc4py import PETSc

from dolfinx import fem, mesh, io, plot, log
from dolfinx.fem import (Constant, dirichletbc, Function, FunctionSpace, Expression )
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from dolfinx.io import XDMFFile
import ufl
from ufl import (TestFunctions, TrialFunction, Identity, grad, det, div, dev, inv, tr, sqrt, conditional , gt, dx, inner, derivative, dot, ln, split)
from datetime import datetime
from dolfinx.plot import vtk_mesh

import pyvista
pyvista.set_jupyter_backend('client')
## Define temporal parameters

## Some parameters

In [2]:
e_dot = 0.001 # strain rate per second 
e_max = 0.01 # absolute value of maximum strain


E = 200e9 # Young’s modulus 
Y0 = 100e6 # Initial yield 
h0 = 0 # Initial hardening modulus 
Y_star = 100e6 # Yield saturation 
r = 0 #Hardening exponent 
C = 0 #Backstress modulus 
gamma = 0 # Nonlinear kinematic parameter



# Define Geometry

In [4]:
length = 10.0 # mm
domain = mesh.create_box(MPI.COMM_WORLD,[[0.0,0.0,0.0],[length,length,length]],[2,2,2],mesh.CellType.tetrahedron)

In [5]:
plotter = pyvista.Plotter()
vtkdata = vtk_mesh(domain, domain.topology.dim)
grid = pyvista.UnstructuredGrid(*vtkdata)
actor = plotter.add_mesh(grid, show_edges=True)
plotter.show()
plotter.close()

Widget(value="<iframe src='http://localhost:61221/index.html?ui=P_0x29f1e2510_1&reconnect=auto' style='width: …

# Some Experiments about Tensor Spaces

In [52]:
# FEniCSx libraries
from dolfinx.fem import FunctionSpace,assemble_scalar,form
from dolfinx.fem.petsc import LinearProblem
from dolfinx.mesh import CellType,create_rectangle
from ufl import SpatialCoordinate,TestFunction,TrialFunction,inner,grad,div,sym,dx,as_vector
import ufl

# Other libraries
from mpi4py import MPI
import numpy as np

# Mesh ------------------------------------------------------------------------

msh = create_rectangle(comm=MPI.COMM_WORLD,
                       points=((0.0,0.0), (1.0,1.0)), n=(8,8),
                       cell_type=CellType.triangle)

x = SpatialCoordinate(msh)

# Variational problem ----------------------------------------------------------

# Some vector field
def ue1(x):
    return ufl.sin(ufl.pi*x[0])*ufl.sin(ufl.pi*x[1])

def ue2(x):
    return ufl.sin(ufl.pi*x[0])*ufl.sin(ufl.pi*x[1])

ue =  as_vector([ue1(x), ue2(x)])

# Tensor space
T_Ele = ufl.TensorElement("CG", msh.ufl_cell(),1,symmetry=True)
V = FunctionSpace(msh,T_Ele)

# Bilinear and linear forms
sig = TrialFunction(V)
tau = TestFunction(V)
a = inner(sig,tau)*dx
L = inner(sym(grad(ue)),tau)*dx

# Linear system
problem = LinearProblem(a,L,bcs=[],petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
wh = problem.solve()


In [69]:
V_sub , map_ing = V.sub(0).collapse()

In [88]:
UVal = Function(V_sub)
UVal.x.array[:]=wh.x.array[map_ing]

In [85]:
UVal.interpolate(P11_expr)

In [86]:
plotter = pyvista.Plotter()
vtkdata = vtk_mesh(msh, msh.topology.dim)
grid = pyvista.UnstructuredGrid(*vtkdata)
grid["mag"] =UVal.x.array
actor = plotter.add_mesh(grid, show_edges=True)

plotter.view_xy()
plotter.show()
plotter.close()

Widget(value="<iframe src='http://localhost:61221/index.html?ui=P_0x2a6792050_6&reconnect=auto' style='width: …